In [1]:
import math
import torch
import gpytorch
import time

import pyro
pyro.enable_validation(True)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
%matplotlib inline

In [2]:
%pdb

Automatic pdb calling has been turned ON


## Introduction

In this notebook, we how to train a functional kernel learning model, training on a simple function. We'll be modeling the function

\begin{align*}
  y &= \sin(2\pi x) + \epsilon \\
  \epsilon &\sim \mathcal{N}(0, 0.2)
\end{align*}

with 11 training examples, and testing on 51 test examples.

For more mathematical detail, please see the paper Functional Distributions over Kernels (NeurIPS 2019).

**Note:** this notebook is not necessarily intended to teach the mathematical background of Gaussian processes, but rather how to train a simple one and make predictions in GPyTorch. For a mathematical treatment, Chapter 2 of Gaussian Processes for Machine Learning provides a very thorough introduction to GP regression (this entire text is highly recommended): http://www.gaussianprocess.org/gpml/chapters/RW2.pdf


#### Set up the GPyTorch Model with Spectral GP kernel
Using the same framework as standard GPyTorch models, we merely drop-in the spectral GP kernel as the covar module

The `initialize_from_data` method does some pre-training on the latent model using the log-periodogram of data as training targets.

For specifics on the components of GPyTorch models we refer to the [GPyTorch Documentation](https://gpytorch.readthedocs.io/en/latest/index.html)

In [3]:
class SpectralModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, **kwargs):
        super(SpectralModel, self).__init__(train_x, train_y, likelihood)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.SpectralGPKernel(**kwargs)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


#### Generate training data and Build GP Model
Generate points in [0, 5] and a sine wave to serve as the response, then split into training and test data.

Pass this data into the GP model from above along with a likelihood

In [4]:
nx = 200
split = 150
full_x = torch.linspace(0, 5, nx).cuda()
full_y = torch.sin(2 * full_x)

train_x = full_x[:split]
train_y = full_y[:split]

test_x = full_x[(split - nx):]
test_y = full_y[(split - nx):]

### Define latent model

In [13]:
class FKL_GPModel(gpytorch.likelihoods.GaussianLikelihood):
    def __init__(self, train_x, train_y, data_likelihood, **kwargs):
        super(FKL_GPModel, self).__init__()
        
        self._max_plate_nesting = 1
        self.data_likelihood = data_likelihood
        self.data_model = SpectralModel(train_x, train_y, self.data_likelihood, 
                                        **kwargs)
        self.train_x = train_x
        self.train_y = train_y
              
    def forward(self, target, input, *params, **kwargs):
        latent_gp = input.mean
        
        self.data_model.covar_module.set_latent_params(latent_gp)
        
        self.data_model.train()
        
        if not self.training:
            self.data_model.eval()
            dist = self.data_model(*params, **kwargs)
        else:
            dist = self.data_likelihood(self.data_model(self.train_x))
        
            with torch.no_grad():
                print('forwards into dist: ', dist.log_prob(self.train_y))
        return dist

In [14]:
#omega = torch.arange(1e-4, 1., 0.01)

data_lh = gpytorch.likelihoods.GaussianLikelihood(noise_prior=gpytorch.priors.SmoothedBoxPrior(1e-8, 1e-4)).cuda()
data_gp = FKL_GPModel(train_x, train_y, data_lh, train_inputs = train_x, register_latent_params=False).cuda()
data_gp.train()
data_lh.train()


GaussianLikelihood(
  (quadrature): GaussHermiteQuadrature1D()
  (noise_covar): HomoskedasticNoise(
    (noise_prior): SmoothedBoxPrior()
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [15]:
omega = data_gp.data_model.covar_module.omega.data.cuda()


In [16]:
from gpytorch.means import QuadraticMean
from gpytorch.priors import NormalPrior
from gpytorch.constraints import LessThan
from gpytorch.kernels import ScaleKernel, MaternKernel, GridKernel
from gpytorch.models import PyroVariationalGP
from gpytorch.distributions import MultivariateNormal
from gpytorch.variational import CholeskyVariationalDistribution, VariationalStrategy

# we construct a Scale(Matern 3/2) kernel with a quadratic mean by default
class LatentGPModel(gpytorch.models.AbstractVariationalGP):
    def __init__(self, train_x, train_y, likelihood, mean_module, covar_module):
        # Define all the variational stuff
        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=int(train_x.numel())
        )
        variational_strategy = VariationalStrategy(
            self, train_x, variational_distribution
        )

        super(LatentGPModel, self).__init__(variational_strategy)
        self.mean_module = mean_module
        self.covar_module = covar_module

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return MultivariateNormal(mean, covar)

# construct by default a LogRBF prior on the latent spectral GP
latent_mean = QuadraticMean()#.to(device)
latent_mean.register_prior(
    "bias_prior",
    prior=NormalPrior(torch.zeros(1), 100.0 * torch.ones(1), transform=None),
    param_or_closure="bias"
)
latent_mean.register_constraint("quadratic_weights", constraint=LessThan(upper_bound=0.0))
latent_mean.register_prior(
    "quadratic_weights_prior",
    prior=NormalPrior(
        torch.zeros(1),
        100.0 * torch.ones(1),
        transform=torch.nn.functional.softplus,
    ),
    param_or_closure="quadratic_weights"
)

latent_covar = ScaleKernel(
    MaternKernel(nu=1.5, lengthscale_prior=NormalPrior(torch.zeros(1), torch.ones(1), transform=torch.exp)),
    outputscale_prior=NormalPrior(torch.zeros(1), torch.ones(1), transform=torch.exp),
)
# latent_covar = GridKernel(latent_covar, grid=self.omega.unsqueeze(-1))

latent_model = LatentGPModel(
    omega, None, data_gp, mean_module=latent_mean, covar_module=latent_covar
).cuda()

In [17]:
data_gp.train();
latent_model.train();

In [18]:
cols = ['red', 'blue', 'green', 'purple', 'orange']
for sample_num in range(5):
    latent_sample = latent_model(omega).sample()
    
    prior_dist = data_gp(latent_sample, train_x)
    prior_preds = prior_dist.sample(torch.Size((3,)))
    
    plt.plot(train_x.cpu(), prior_preds.data.cpu().t().numpy(), color = cols[sample_num], alpha = 0.3)

RuntimeError: The size of tensor a (50) must match the size of tensor b (150) at non-singleton dimension 0

> /home/wesley/anaconda3/lib/python3.7/site-packages/torch/functional.py(52)broadcast_tensors()
     50                 [0, 1, 2]])
     51     """
---> 52     return torch._C._VariableFunctions.broadcast_tensors(tensors)
     53 
     54 

ipdb> u
> /home/wesley/anaconda3/lib/python3.7/site-packages/torch/distributions/utils.py(33)broadcast_all()
     31         values = [v if torch.is_tensor(v) else torch.tensor(v, **options)
     32                   for v in values]
---> 33     return torch.broadcast_tensors(*values)
     34 
     35 

ipdb> u
> /home/wesley/anaconda3/lib/python3.7/site-packages/torch/distributions/normal.py(44)__init__()
     42 
     43     def __init__(self, loc, scale, validate_args=None):
---> 44         self.loc, self.scale = broadcast_all(loc, scale)
     45         if isinstance(loc, Number) and isinstance(scale, Number):
     46             batch_shape = torch.Size()

ipdb> u
> /home/wesley/Documents/Code/wjm_gpytorch/gpytorch/likelihoods/gaussian_likeliho

ipdb> u
> <ipython-input-18-9b3844953ee7>(5)<module>()
      3     latent_sample = latent_model(omega).sample()
      4 
----> 5     prior_dist = data_gp(latent_sample, train_x)
      6     prior_preds = prior_dist.sample(torch.Size((3,)))
      7 

ipdb> latent_sample
tensor([ 0.9143,  0.8088,  0.4365,  0.1067,  0.1153, -0.1164, -0.3102,  0.1845,
         0.7008,  1.3262,  1.3993,  1.0417,  0.8893,  0.7571,  0.7995,  0.0605,
        -0.1652,  0.2223,  0.7226,  0.6930,  0.4755,  0.8699,  1.2511,  1.4827,
         1.0610,  0.5035,  0.6980,  0.7145,  1.3426,  1.1889,  0.7820,  0.4238,
         0.6127,  0.8161,  0.8802,  0.8996,  0.9877,  1.7074,  1.9940,  2.3819,
         1.9741,  1.7561,  2.0416,  2.4312,  1.9133,  2.0925,  2.0627,  2.3866,
         2.3666,  1.7558], device='cuda:0')
ipdb> q


In [11]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_x, train_y), 
                                           batch_size = 150, shuffle = True)

In [12]:
latent_model.train()
data_gp.train()

# We'll do 6 epochs of training in this tutorial
num_epochs = 40

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': latent_model.parameters()},
    {'params': data_gp.parameters()},
], lr=0.01)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(data_gp, latent_model, num_data=train_y.size(0), combine_terms=False)

# We use more CG iterations here because the preconditioner introduced in the NeurIPS paper seems to be less
# effective for VI.
with gpytorch.settings.max_cg_iterations(45):
    for i in range(num_epochs):
        scheduler.step()
        # Within each iteration, we will go over each minibatch of data
        for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):
            start_time = time.time()
            optimizer.zero_grad()
            output = latent_model(omega)
            # with combine_terms=False, we get the terms of the ELBO separated so we can print them individually if we'd like.
            # loss = -mll(output, y_batch) would also work.
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

            loss.backward()
            optimizer.step()

/home/wesley/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


RuntimeError: The size of tensor a (150) must match the size of tensor b (50) at non-singleton dimension 0

> /home/wesley/Documents/Code/wjm_gpytorch/gpytorch/likelihoods/gaussian_likelihood.py(84)expected_log_prob()
     82         noise = self.noise_covar.noise
     83 
---> 84         res = ((target - mean) ** 2 + variance) / noise + noise.log() + math.log(2 * math.pi)
     85         return res.mul(-0.5).sum(-1)
     86 

ipdb> ll
     80     def expected_log_prob(self, target: Tensor, input: MultivariateNormal, *params: Any, **kwargs: Any) -> Tensor:
     81         mean, variance = input.mean, input.variance
     82         noise = self.noise_covar.noise
     83 
---> 84         res = ((target - mean) ** 2 + variance) / noise + noise.log() + math.log(2 * math.pi)
     85         return res.mul(-0.5).sum(-1)
     86 

ipdb> target
tensor([-0.9981,  0.9144,  0.5470,  0.2953,  0.4354, -0.9713, -0.3462, -0.8709,
         0.1484,  0.9653,  0.6844, -0.2014, -0.8299, -0.5266,  0.0260,  0.1997,
        -0.8817,  0.4151, -0.7698, -0.1741, -0.4386,  0.9151,  0.2486, -0.9426,
         0.0502, -0

In [23]:
# train the model
from pyro import optim

base_lr = 1e-2
iter_print = 5

print('Basic lr for most of parameters: {}'.format(base_lr))

# set learning rates for different hyperparameters
def per_param_callable(module_name, param_name):
    if param_name == 'covar_module.base_kernel.raw_lengthscale':
        return {"lr": 0.1}
    elif param_name == 'variational_strategy.variational_distribution.variational_mean':
        return {"lr": base_lr}
    else:
        return {"lr": base_lr}

# Use the adam optimizer
optimizer = optim.Adam(per_param_callable)

pyro.clear_param_store() # clean run

losses, rbf, means = [], [], []

means.append(latent_model.variational_strategy.variational_distribution.\
             variational_mean.detach().cpu().numpy()) #save initial mean

#def train(num_iter=15):
num_iter = 15
elbo = pyro.infer.Trace_ELBO(num_particles=1, vectorize_particles=True)
svi = pyro.infer.SVI(latent_model.model, latent_model.guide, optimizer, elbo)
latent_model.train()

for i in range(num_iter):
    latent_model.zero_grad()
    loss = svi.step(omega, train_y)
    print(latent_model(omega).log_prob(data_gp.data_model.covar_module.latent_params).sum(), 
         data_gp.data_model.covar_module.latent_params.norm())
    losses.append(loss)
    rbf.append(latent_model.covar_module.base_kernel.lengthscale.item())
    if not (i + 1) % iter_print:
        print('Iter {}/{} - Loss: {:.3}   lengthscale: {:.3}'.format(
            i + 1, num_iter, loss,
            latent_model.covar_module.base_kernel.lengthscale.item(),
        ))
        means.append(latent_model.variational_strategy.variational_distribution.\
                     variational_mean.detach().cpu().numpy())
        
#%time train()

Basic lr for most of parameters: 0.01
forwards into dist:  tensor(-271.0613, device='cuda:0')
forwards into dist:  tensor(-283.2576, device='cuda:0')
tensor(-44.7022, device='cuda:0', grad_fn=<SumBackward0>) tensor(7.8205, device='cuda:0', grad_fn=<NormBackward0>)
forwards into dist:  tensor(-297.0042, device='cuda:0')
tensor(-42.5959, device='cuda:0', grad_fn=<SumBackward0>) tensor(8.3975, device='cuda:0', grad_fn=<NormBackward0>)
forwards into dist:  tensor(-290.5873, device='cuda:0')
tensor(-57.0369, device='cuda:0', grad_fn=<SumBackward0>) tensor(9.2815, device='cuda:0', grad_fn=<NormBackward0>)
forwards into dist:  tensor(-276.0399, device='cuda:0')
tensor(-68.8978, device='cuda:0', grad_fn=<SumBackward0>) tensor(8.3217, device='cuda:0', grad_fn=<NormBackward0>)
forwards into dist:  tensor(-283.0795, device='cuda:0')
tensor(-65.2813, device='cuda:0', grad_fn=<SumBackward0>) tensor(8.9567, device='cuda:0', grad_fn=<NormBackward0>)
Iter 5/15 - Loss: 1.42e+04   lengthscale: 0.607
for

In [28]:
for obj in elbo._get_traces(latent_model.model, latent_model.guide):
    print(obj)

TypeError: guide() missing 2 required positional arguments: 'input' and 'output'

> /home/wesley/anaconda3/lib/python3.7/site-packages/pyro/infer/elbo.py(132)wrapped_fn()
    130         def wrapped_fn(*args, **kwargs):
    131             if self.num_particles == 1:
--> 132                 return fn(*args, **kwargs)
    133             with pyro.plate("num_particles_vectorized", self.num_particles, dim=-self.max_plate_nesting):
    134                 return fn(*args, **kwargs)

ipdb> q


In [ ]:
# prot loss function and kernel length
fig, (loss, kern) = plt.subplots(1,2,figsize=(12,2))
loss.plot(losses)
loss.set_xlabel("Epoch")
loss.set_ylabel("Loss")
kern.plot(rbf)
kern.set_xlabel("Epoch")
kern.set_ylabel("Kernel scale parameter")
fig.show()

In [ ]:
test_x = torch.linspace(0, 15, 500)

In [ ]:
data_gp.eval()

latent_model.eval()

with torch.no_grad():
    output_dist = latent_model(omega)
    
samples = output_dist.sample(torch.Size((20,)))

In [ ]:
list(latent_model.named_parameters())

In [ ]:
for i, sample in enumerate(samples):
    if i is 0:
        plt.plot(omega, sample.exp(), color = 'black', alpha = 0.3, label = 'S(w)')
    else:
        plt.plot(omega, sample.exp(), color = 'black', alpha = 0.3)

plt.xlabel('omega')
plt.ylabel('S(omega)')
plt.grid()
plt.legend()

In [ ]:
for i, sample in enumerate(samples):
    with gpytorch.settings.fast_pred_var():
        pred_dist = data_gp(sample.data, test_x)
    
    pred_mean = pred_dist.mean.data.numpy()
    lower, upper = pred_dist.confidence_region()
    
    if i is 0:
        plt.plot(test_x, pred_mean, color = 'blue', label = 'Predictive Mean')
    else:
        plt.plot(test_x, pred_mean, color = 'blue')
    
    plt.fill_between(test_x.numpy(), lower.data.numpy(), upper.data.numpy(), alpha = 0.03, color = 'blue')
    
plt.plot(train_x, train_y, marker = 'x', color = 'black', label = 'Training Data')
plt.grid()
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

In [ ]:
covar_output = data_gp.data_model.covar_module(torch.zeros(1), torch.linspace(0., 20., 100)).evaluate()

In [ ]:
plt.plot(torch.linspace(0., 20., 100), covar_output.view(-1).numpy())

In [ ]:
covar_output